In [2]:
!pip install earthengine-api --upgrade
!pip install folium
!pip install geehydro
!pip install geemap

In [3]:
import ee
import folium
import geehydro
import geemap

from datetime import datetime as dt
from IPython.display import Image

In [4]:
ee.Authenticate()

Enter verification code: 4/1AY0e-g4_inJzh4GYpEvENL5oj2Kz7ye38yxjNVK9wtHXFL_HqKcXr81eunA

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[41.100618, 29.024453], zoom_start=15)
Ituna_map

In [7]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

In [8]:
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([41.0, 29.0,
                                   41.5, 29.5])

In [9]:
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

In [10]:
# choose dates
landsat = landsat.filterDate('2020-05-01','2021-01-01')

In [11]:
landsat_AOI.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'LANDSAT/LC08/C01/T1_SR',
 'version': 1608884393106071,
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'visualization_1_bands': 'B5,B4,B3',
  'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png',
  'visualization_1_max': '30000.0',
  'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LASRC_ProductGuide-v3.pdf">LaSRC</a>\nand includes a cloud, shadow, water and snow mask produced 

In [12]:
print('Total number:', landsat_AOI.size().getInfo())

Total number: 516


In [13]:
# the names of each Landsat 8 band
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [14]:
# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

Cloud Cover (%): 0


In [15]:
# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))
time = date.getInfo()['value']/1000.
dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

'2013-05-14 07:42:57'

In [16]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
Image(url = least_cloudy.getThumbUrl(parameters))

In [52]:
#Panchromatic NDVI
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])
Ituna_map.addLayer(ndvi)
Ituna_map

In [53]:
#NDVI with Palette
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}

Ituna_map.addLayer(ndvi, ndvi_parameters)
Ituna_map